In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) |

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

### Objective

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses.

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

### Costs
This tutorial uses billable components of Google Cloud:

* Vertex AI Generative AI Studio

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Install Vertex AI SDK

In [ ]:
!pip install google-cloud-aiplatform protobuf==3.19.3 --upgrade --user

In [ ]:
!pip install -U google-cloud-aiplatform "shapely<2"

**Colab only:** Run the following cell to restart the kernel or use the button to restart the kernel. For **Vertex AI Workbench** you can restart the terminal using the button on top.

In [1]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Authenticating your notebook environment

- If you are using **Colab** to run this notebook, run the cell below and continue.
- If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [16]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

- If you are running this notebook in a local development environment:
  - Install the [Google Cloud SDK](https://cloud.google.com/sdk).
  - Obtain authentication credentials. Create local credentials by running the following command and following the oauth2 flow (read more about the command [here](https://cloud.google.com/sdk/gcloud/reference/beta/auth/application-default/login)):

    ```bash
    gcloud auth application-default login
    ```

### Import libraries

**Colab only:** Run the following cell to initialize the Vertex AI SDK. For Vertex AI Workbench, you don't need to run this.

In [19]:
import vertexai

PROJECT_ID = "qwiklabs-gcp-01-7ce8fed2598d"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

vertexai.init(project=PROJECT_ID, location=REGION)

In [20]:
from vertexai.language_models import TextGenerationModel
from vertexai.language_models import ChatModel

### Load model

In [21]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [22]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers? Thank you!"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

* **Dried & Lovely**
* **Dried Blooms**
* **Dried Florals**
* **Dried Arrangements**
* **Preserved Petals**
* **Forever Flowers**
* **Timeless Blooms**
* **Dried Flower Boutique**
* **Dried Flower Shop**


✅ Recommended. The prompt below is to the point and concise.

In [23]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

* **Dried Blooms**
* **Preserved Petals**
* **Forever Flowers**
* **Dried & Wild**
* **Naturally Beautiful**
* **Blooming Memories**
* **Sentimental Bouquets**
* **Timeless Treasures**


### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 Not recommended. The prompt below is too generic.

In [24]:
prompt = "Tell me about Earth"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

Earth is the third planet from the Sun, and the only astronomical object known to harbor life. It is the densest and fifth-largest of the eight planets in the Solar System. Earth's atmosphere is composed of 78% nitrogen, 21% oxygen, 1% other gases, and water vapor. The Earth's surface is divided into several tectonic plates that move around the planet's surface. The Earth's interior is divided into a solid inner core, a liquid outer core, and a mantle. The Earth's magnetic field is generated by the motion of the liquid outer core. The Earth's orbit around the Sun takes 365.256 days, or one year. The Earth's axis is tilted at an angle of 23.5 degrees, which causes the seasons. The Earth's rotation period is 24 hours, or one day.


✅ Recommended. The prompt below is specific and well-defined.

In [25]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

* **Earth is the only planet known to support life.** This is due to a number of factors, including its distance from the sun, its atmosphere, and its water.
* **Earth has a large moon.** The moon is thought to have played a role in the development of life on Earth, by stabilizing the planet's rotation and providing a source of tides.
* **Earth has a relatively thin atmosphere.** This atmosphere is made up of nitrogen, oxygen, and argon, and it protects the planet from harmful radiation from the sun.
* **Earth has a large amount of water.** Water is essential for life, and it covers about 70% of the Earth's surface.
* **Earth has a variety of landforms.** These landforms include mountains, valleys, plains, and deserts. They provide a variety of habitats for life.
* **Earth has a variety of climate zones.** These climate zones range from the frigid Arctic to the hot tropics. They support a wide variety of plant and animal life.

These are just a few of the ways that make Earth unique co

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [26]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

The best method of boiling water is to use a kettle. A kettle is a pot with a spout that is designed specifically for boiling water. It has a lid that helps to keep the heat in and prevents the water from boiling over.

The sky is blue because of a phenomenon called Rayleigh scattering. This is the scattering of light by particles that are smaller than the wavelength of light. The particles in the atmosphere that cause Rayleigh scattering are molecules of nitrogen and oxygen.

When sunlight hits these molecules, the blue light is scattered in all directions. The other colors of light are scattered less, so they do not reach our eyes as much. This is why the sky appears blue during the day.


✅ Recommended. The prompts below asks one task a time.

In [27]:
prompt = "What's the best method of boiling water?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

The best method of boiling water is to use a kettle. A kettle is a pot with a spout that is designed specifically for boiling water. It is typically made of metal, and has a lid that fits snugly to prevent steam from escaping. Kettles come in a variety of sizes, so you can choose one that is right for your needs. To boil water in a kettle, simply fill it with water and place it on the stove. Turn the heat to high, and wait for the water to come to a boil. Once the water is boiling, turn off the heat and carefully pour the water into your desired container.


In [28]:
prompt = "Why is the sky blue?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

The sky is blue because of a phenomenon called Rayleigh scattering. This is the scattering of light by particles that are smaller than the wavelength of light. In the case of the sky, the particles that are doing the scattering are molecules of nitrogen and oxygen.

When sunlight hits these molecules, the blue light is scattered more than the other colors. This is because the blue light has a shorter wavelength than the other colors. The red light, orange light, and yellow light are not scattered as much, so they travel through the atmosphere to our eyes. This is why we see the sky as blue.

The amount of scattering depends on the wavelength of light and the size of the particles. The shorter the wavelength of light, the more it is scattered. The smaller the particles, the more they scatter light.

This is why the sky is blue during the day. However, at sunrise and sunset, the sunlight has to travel through more of the atmosphere to reach our eyes. This means that more of the blue ligh

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn’t a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect. 

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. You may notice that sometimes it will confidently, but inaccurately, say "The first elephant to visit the moon was Luna".

In [29]:
prompt = "Who was the first elephant to visit the moon?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

The first elephant to visit the moon was Luna. Luna was a female elephant who was born in 1963 at the San Diego Zoo. In 1968, she was selected to be the first elephant to visit the moon. Luna was trained to wear a spacesuit and to ride in a rocket. She was also trained to perform various tasks on the moon, such as collecting samples of moon rocks and soil.

Luna's trip to the moon was a success. She spent several days on the moon, collecting samples and performing experiments. She also became the first animal to walk on the moon. Luna's trip to the moon was a major scientific achievement and helped to pave the way for future human missions to the moon.

Luna returned to Earth in 1969. She was a national hero and was celebrated by people all over the world. Luna lived a long and happy life, dying in 2003 at the age of 40.


Clearly the chatbot is hallucinating since no elephant has ever flown to the moon. But how do we prevent these kinds of inappropriate questions and more specifically, reduce hallucinations? 

There is one possible method called the Determine Appropriate Response (DARE) prompt, which cleverly uses the LLM itself to decide whether it should answer a question based on what its mission is.

Let's see how it works by creating a chatbot for a travel website with a slight twist.

In [30]:
chat_model = ChatModel.from_pretrained("chat-bison@002")

chat = chat_model.start_chat()
dare_prompt = """Remember that before you answer a question, you must check to see if it complies with your mission.
If not, you can say, Sorry I can't answer that question."""

print(
    chat.send_message(
        f"""
Hello! You are an AI chatbot for a travel web site.
Your mission is to provide helpful queries for travelers.

{dare_prompt}
"""
    )
)

MultiCandidateTextGenerationResponse(text=" Hello! I'm here to help you plan your next trip. Whether you're looking for a relaxing beach vacation or an adventurous city getaway, I can help you find the perfect destination and activities for your budget and interests.", _prediction_response=Prediction(predictions=[{'candidates': [{'author': '1', 'content': " Hello! I'm here to help you plan your next trip. Whether you're looking for a relaxing beach vacation or an adventurous city getaway, I can help you find the perfect destination and activities for your budget and interests."}], 'groundingMetadata': [{}], 'safetyAttributes': [{'categories': ['Finance', 'Insult', 'Profanity', 'Sexual'], 'safetyRatings': [{'severityScore': 0.1, 'probabilityScore': 0.1, 'severity': 'NEGLIGIBLE', 'category': 'Dangerous Content'}, {'probabilityScore': 0.1, 'severity': 'NEGLIGIBLE', 'severityScore': 0.0, 'category': 'Harassment'}, {'probabilityScore': 0.0, 'severity': 'NEGLIGIBLE', 'category': 'Hate Speech

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [ ]:
prompt = "What is the best place for sightseeing in Milan, Italy?"
print(chat.send_message(prompt))

Now let us pretend to be a not-so-nice user and ask the chatbot a question that is unrelated to travel.

In [ ]:
prompt = "Who was the first elephant to visit the moon?"
print(chat.send_message(prompt))

You can see that the DARE prompt added a layer of guard rails that prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [ ]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [ ]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn Javascript
c) learn Fortran
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.